<a href="https://colab.research.google.com/github/DonPex/RedditApiNotebooks/blob/main/top_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip3 install praw

In [ ]:
import praw
from textblob import TextBlob
# VADER can be accessed by the NLTK library.
import nltk
# Download the VADER tool and access it through the NLTK library.
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
id = '<retrieve_id_from_reddit>'
secret = '<retrieve_secret_from_reddit>'

reddit_api = praw.Reddit(client_id=id,
                         client_secret=secret,
                         user_agent='testscript by u/<your_username>')


## Top N post (upvotes) for a subreddit

In [6]:
subr_name = 'italiapersonalfinance'
period = 'month'
n = 5

top_posts = reddit_api.subreddit(subr_name).top(period, limit=n)

for submission in top_posts:
     print("Title: {} - Ups: {} - UpvoteRatio: {} - URL: {}".format(submission.title, submission.ups, submission.upvote_ratio, submission.url))


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Title: Il lavoro che consuma e uccide. - Ups: 285 - UpvoteRatio: 0.98 - URL: https://www.reddit.com/r/ItaliaPersonalFinance/comments/r49t7d/il_lavoro_che_consuma_e_uccide/
Title: Cari sviluppatori: smettete di cercare lavoro in Italia! - Ups: 188 - UpvoteRatio: 0.87 - URL: https://www.reddit.com/r/ItaliaPersonalFinance/comments/rb44vu/cari_sviluppatori_smettete_di_cercare_lavoro_in/
Title: Una birra per u/Xhuly - Ups: 100 - UpvoteRatio: 0.93 - URL: https://www.reddit.com/r/ItaliaPersonalFinance/comments/r516u2/una_birra_per_uxhuly/
Title: Sto ultimando lo sviluppo(90%) di un'app di budgeting diversa dalle solite. Proponete delle feature che vorreste; le integrerò e vi manterrò aggiornati sugli sviluppi! - Ups: 87 - UpvoteRatio: 0.97 - URL: https://www.reddit.com/r/ItaliaPersonalFinance/comments/qrui42/sto_ultimando_lo_sviluppo90_di_unapp_di_budgeting/
Title: 60 ore di straordinari al mese sono troppo poche - Ups: 78 - UpvoteRatio: 0.96 - URL: https://www.reddit.com/r/ItaliaPersonalFina

## Sentiment Analysis with NLTK Vader

In [7]:
def replies_of(top_level_comment,
               count_comment,
               sub_entries_textblob,
               sub_entries_nltk):
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                #print('-' * count_comment, comment.body)
                text_blob_sentiment(comment.body,
                                       sub_entries_textblob)
                nltk_sentiment(comment.body, sub_entries_nltk)
            except:
               continue
            replies_of(comment, 
                       count_comment,
                       sub_entries_textblob,
                       sub_entries_nltk)
            

def text_blob_sentiment(review, sub_entries_textblob):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity >= 0.0001:
        if analysis.sentiment.polarity > 0:
            sub_entries_textblob['positive'] = sub_entries_textblob['positive']+1
            return 'Positive'
    elif analysis.sentiment.polarity <= -0.0001:
        if analysis.sentiment.polarity <= 0:
            sub_entries_textblob['negative'] = sub_entries_textblob['negative'] + 1
            return 'Negative'
    else:
        sub_entries_textblob['neutral'] = sub_entries_textblob['neutral'] + 1
        return 'Neutral'

sia = SentimentIntensityAnalyzer()
def nltk_sentiment(review, sub_entries_nltk):
    vs = sia.polarity_scores(review)
    if not vs['neg'] > 0.05:
        if vs['pos'] - vs['neg'] > 0:
            sub_entries_nltk['positive']=sub_entries_nltk['positive'] + 1
            return 'Positive'
        else:
            sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    elif not vs['pos'] > 0.05:
        if vs['pos'] - vs['neg'] <= 0:
            sub_entries_nltk['negative']=sub_entries_nltk['negative'] + 1
            return 'Negative'
        else:
            sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    else:
        sub_entries_nltk['neutral']=sub_entries_nltk['neutral'] + 1
        return 'Neutral'

In [24]:
top_posts = reddit_api.subreddit('cryptocurrency').top('month', limit=5)

for submission in top_posts:
    sub_entries_textblob = {'negative': 0, 'positive' : 0, 'neutral' : 0}
    sub_entries_nltk = {'negative': 0, 'positive' : 0, 'neutral' : 0}
    print('Title of the post :', submission.title)
    print('URL : ', submission.url)

    text_blob_sentiment(submission.title, sub_entries_textblob)
    nltk_sentiment(submission.title, sub_entries_nltk)
    #print("\n")
    submission_comm = reddit_api.submission(id=submission.id)

    for count, top_level_comment in enumerate(submission_comm.comments):
        # print(f"-------------{count} top level comment start--------------")
        count_comm = 0
        try :
            # print(top_level_comment.body)
            text_blob_sentiment(top_level_comment.body, sub_entries_textblob)
            nltk_sentiment(top_level_comment.body, sub_entries_nltk)
            replies_of(top_level_comment,
                        count_comm,
                        sub_entries_textblob,
                        sub_entries_nltk)
        except:
            continue
    print('Over all Sentiment of Topic by TextBlob :', sub_entries_textblob)
    print('Over all Sentiment of Topic by VADER :', sub_entries_nltk)
    print("\n\n\n")

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Title of the post : AOC reveals she doesn't hold bitcoin because she wants to be an unbiased lawmaker
URL :  https://markets.businessinsider.com/news/currencies/aoc-bitcoin-crypto-investment-unbiased-lawmaker-house-financial-services-committee-2021-12


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Over all Sentiment of Topic by TextBlob : {'negative': 90, 'positive': 264, 'neutral': 146}
Over all Sentiment of Topic by VADER : {'negative': 79, 'positive': 169, 'neutral': 252}




Title of the post : Tether (USDT) created $1,500,000,000 Worth of USDT Out of Thin Air in the Last 24h: Nothing of it is backed by actual Cash
URL :  https://www.reddit.com/r/CryptoCurrency/comments/r9fni8/tether_usdt_created_1500000000_worth_of_usdt_out/


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Over all Sentiment of Topic by TextBlob : {'negative': 118, 'positive': 190, 'neutral': 191}
Over all Sentiment of Topic by VADER : {'negative': 94, 'positive': 139, 'neutral': 266}




Title of the post : Predictions Tournament
URL :  https://reddit.com/r/CryptoCurrency/predictions?tournament=tnmt-ddb5d4f9-58ab-47da-a031-f3e2ad0b71bf


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Over all Sentiment of Topic by TextBlob : {'negative': 36, 'positive': 118, 'neutral': 130}
Over all Sentiment of Topic by VADER : {'negative': 40, 'positive': 98, 'neutral': 146}




Title of the post : It's been almost two months since the pandora papers unveiled a massive network of the rich and powerful hoarding billions of dollars in offshare tax havens, but instead of addressing the obvious fraud at hand, politicians aim to make crypto the "bad guy". You can't make this up.
URL :  https://www.reddit.com/r/CryptoCurrency/comments/r5o83b/its_been_almost_two_months_since_the_pandora/


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Over all Sentiment of Topic by TextBlob : {'negative': 142, 'positive': 198, 'neutral': 157}
Over all Sentiment of Topic by VADER : {'negative': 106, 'positive': 121, 'neutral': 270}




Title of the post : ETH is bad, and I am tired of pretending it's not.
URL :  https://www.reddit.com/r/CryptoCurrency/comments/qw2e32/eth_is_bad_and_i_am_tired_of_pretending_its_not/
Over all Sentiment of Topic by TextBlob : {'negative': 105, 'positive': 218, 'neutral': 173}
Over all Sentiment of Topic by VADER : {'negative': 73, 'positive': 152, 'neutral': 271}




